In [1]:
from GAE import GAE
from CAE import Autoencoder
import Get_data as Gd
import torch
import torch.nn as nn
from pytorch_msssim import ssim

import numpy as np
import scipy
import matplotlib.pyplot as plt

/home/scheng1/miniconda3/envs/wl4023/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Get test data

In [3]:
xyfile = "/data/Sibo_22Mar2024/case_0/hessian_/xy_coords.npy"
pos = torch.tensor(np.load(xyfile), dtype=torch.float32)

In [4]:
sparse_graph = scipy.sparse.load_npz('/data/Sibo_22Mar2024/sparse_matrix_fixed.npz')
indices = np.array(sparse_graph.nonzero())
values = sparse_graph.data
shape = sparse_graph.shape

# sparse edge tensor
edge_tensor = torch.sparse_coo_tensor(indices, values, shape, dtype=torch.float).coalesce()
edge_index = edge_tensor.indices()
edge_weight = edge_tensor.values()
print(edge_tensor.shape)
print(edge_index)
print(edge_weight)

torch.Size([97149, 97149])
tensor([[    0,     0,     1,  ..., 97148, 97148, 97148],
        [    1,     3,     0,  ..., 97145, 97146, 97147]])
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [5]:
folders = ['/data/Sibo_22Mar2024/case_18/hessian_']
test_np, length = Gd.get_all_nodes(folders[-1:])
test_np = np.expand_dims(test_np[:, :, 0], axis=2)
test_tensor = torch.tensor(test_np, dtype=torch.float)
test_tensor_set = Gd.MyDataset(test_tensor)

# Load model

In [6]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

class NRMSE(nn.Module):
    def __init__(self, total_dataset):
        super(NRMSE, self).__init__()
        self.rmse = RMSELoss()
        self.factor = total_dataset.max()-total_dataset.min()

    def forward(self, y_true, y_pred):
        return self.rmse(y_true, y_pred)/self.factor

## Load GAE

In [7]:
lengthscales = [0.17, 1]  # 例如定义的长度比例
num_mp_layers = [2, 3]  # 例如定义的消息传递层数

bounding_box = []
if len(lengthscales) > 0:
    x_lo = pos[:,0].min() - lengthscales[0]/2
    x_hi = pos[:,0].max() + lengthscales[0]/2
    y_lo = pos[:,1].min() - lengthscales[0]/2
    y_hi = pos[:,1].max() + lengthscales[0]/2
    bounding_box = [x_lo.cpu(), x_hi.cpu(), y_lo.cpu(), y_hi.cpu()]

In [8]:
gae = GAE(1, 1, bounding_box, lengthscales, num_mp_layers, 16, 2)
criterion = RMSELoss()
metric = NRMSE(test_tensor)
# 加载保存的状态字典
state_dict = torch.load('/home/scheng1/wl4023/IRP/BaselineModels/GAE/GAE.pth')
gae.load_state_dict(state_dict)

1 16
16 1
1 16
16 1


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Load CAE

In [ ]:
cae = Autoencoder(1, 49)
# 加载保存的状态字典
state_dict = torch.load('/home/scheng1/wl4023/IRP/BaselineModels/CNN_LSTM/CAE.pth')
cae.load_state_dict(state_dict)
# model.to(device)

<All keys matched successfully>

# Prediction

In [ ]:
criterion = RMSELoss()
metric = NRMSE(test_tensor)

## GAE

In [ ]:
GAE_output = torch.zeros_like(test_tensor)
GAE_RMSEs = []
GAE_NRMSEs = []


for i in range(len(test_tensor_set)):
    GAE_output[i], _, _ = gae(test_tensor_set[i], edge_index, edge_weight, pos)

    rmse = criterion(test_tensor_set[i], GAE_output[i]).item()
    nrmse = metric(test_tensor_set[i], GAE_output[i]).item()

    GAE_RMSEs.append(rmse)
    GAE_NRMSEs.append(nrmse)

GAE_RMSEs = np.array(GAE_RMSEs)
GAE_NRMSEs = np.array(GAE_NRMSEs)

print(f"RMSE Loss: {GAE_RMSEs.mean()}, NRMSE Loss: {GAE_NRMSEs.mean()}")

RMSE Loss: 7.34042721748352, NRMSE Loss: 0.17132924497127533


## CAE

In [ ]:
CAE_output = torch.zeros_like(test_tensor)
CAE_RMSEs = []
CAE_NRMSEs = []


for i in range(len(test_tensor_set)):
    x = test_tensor_set[i].permute(1, 0)
    output = cae(x)

    CAE_output[i] = output.permute(0, 2, 1)

    rmse = criterion(test_tensor_set[i], CAE_output[i]).item()
    nrmse = metric(test_tensor_set[i], CAE_output[i]).item()

    CAE_RMSEs.append(rmse)
    CAE_NRMSEs.append(nrmse)

CAE_RMSEs = np.array(CAE_RMSEs)
CAE_NRMSEs = np.array(CAE_NRMSEs)

print(f"RMSE Loss: {CAE_RMSEs.mean()}, NRMSE Loss: {CAE_NRMSEs.mean()}")

RMSE Loss: 5.619884696006775, NRMSE Loss: 0.1311709214001894


## calculate ssim

In [ ]:
origin_data = test_tensor.unsqueeze(1)
GAE_data = GAE_output.unsqueeze(1)
CAE_data = CAE_output.unsqueeze(1)

In [ ]:
# 检查数据范围
original_min = origin_data.min().item()
original_max = origin_data.max().item()
cae_min = CAE_data.min().item()
cae_max = CAE_data.max().item()
gae_min = GAE_data.min().item()
gae_max = GAE_data.max().item()

print(f"Original image range: {original_min} to {original_max}")
print(f"CAE reconstructed image range: {cae_min} to {cae_max}")
print(f"GAE reconstructed image range: {gae_min} to {gae_max}")

# 计算数据范围
global_min = min(original_min, cae_min, gae_min)
global_max = max(original_max, cae_max, gae_max)
data_range = global_max - global_min

Original image range: 4.794646263122559 to 47.63862991333008
CAE reconstructed image range: 4.008631706237793 to 51.62077331542969
GAE reconstructed image range: 16.50448226928711 to 45.24418258666992


In [ ]:
GAE_ssim_values = []
CAE_ssim_values = []

for i in range(len(test_tensor_set)):
    CAE_sample = CAE_data[i].view(1, 1, 97149, 1)  # 变成 (1, 1, 97149, 1)
    GAE_sample = GAE_data[i].view(1, 1, 97149, 1)  # 变成 (1, 1, 97149, 1)
    origin_sample = origin_data[i].view(1, 1, 97149, 1)  # 变成 (1, 1, 97149, 1)

    print(CAE_sample.shape)
    ssim_cae = ssim(CAE_sample, origin_sample, data_range=data_range, size_average=True)
    ssim_gae = ssim(GAE_sample, origin_sample, data_range=data_range, size_average=True)

    CAE_ssim_values.append(ssim_cae)
    GAE_ssim_values.append(ssim_gae)

CAE_ssim_values = np.array(CAE_ssim_values)
GAE_ssim_values = np.array(GAE_ssim_values)

torch.Size([1, 1, 97149, 1])


ValueError: Input images should be 4-d or 5-d tensors, but got torch.Size([1, 1, 97149])

: 

In [ ]:
fig1, axs = plt.subplots(1, 2, figsize=(12, 6))

x = np.arange(0, 100)

axs[0].scatter(x, CAE_ssim_values, label='CAE ssim')
axs[0].scatter(x, GAE_ssim_values, label='GAE ssim')
axs[0].set_title("SSIM")
axs[0].legend()

axs[1].scatter(x, CAE_RMSEs, label='CAE RMSE')
axs[1].scatter(x, GAE_RMSEs, label='GAE RMSE')
axs[1].set_title("NRMSE Loss")
axs[1].legend()


plt.tight_layout()  # 调整布局以防止重叠
plt.show()

In [ ]:
print(origin_data.shape, GAE_data.shape, CAE_data.shape)

torch.Size([100, 1, 97149, 1]) torch.Size([100, 1, 97149, 1]) torch.Size([100, 1, 97149, 1])
